In [1]:
import tensorflow.compat.v1 as tf
tf.disable_eager_execution()
tf.test.gpu_device_name()

'/device:GPU:0'

In [2]:
from google.colab import files
files.upload()
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!kaggle datasets download -d uciml/sms-spam-collection-dataset
!unzip sms-spam-collection-dataset.zip

Saving kaggle.json to kaggle.json
  0% 0.00/211k [00:00<?, ?B/s]
100% 211k/211k [00:00<00:00, 26.6MB/s]
Archive:  sms-spam-collection-dataset.zip
  inflating: spam.csv                


In [3]:
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from tqdm.notebook import tqdm
nltk.download('punkt')
nltk.download('stopwords')
tokenizer = RegexpTokenizer(r'\w+')
tqdm.pandas()

MAX_LEN = 128
def make_clean(s) :
  for i in range(10) :
    s = s.replace(str(i), ' ')
  tokens = np.array(tokenizer.tokenize(s.lower()))
  tokens = tokens[~np.isin(tokens, stopwords.words())]
  return ' '.join(tokens)

df = pd.read_csv('spam.csv', encoding = "ISO-8859-1")
df['clean'] = df['v2'].progress_apply(make_clean)
df['label'] = (df['v1']=='ham').astype(int)

random_index = np.zeros(df.shape[0]).astype(bool)
random_index[np.random.choice(df.shape[0], int(0.2*df.shape[0]))] = True
train_df, test_df = df[~random_index], df[random_index]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [4]:
train_df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4,clean,label
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN,go jurong point crazy available bugis great wo...,1
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN,free entry wkly comp win fa cup final tkts st ...,0
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN,dun say early hor already say,1
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN,think goes usf lives around though,1
5,spam,FreeMsg Hey there darling it's been 3 week's n...,NaN,NaN,NaN,freemsg hey darling week word back like fun st...,0


In [0]:
import numpy as np

MAX_LEN = 128

# Create datasets (Only take up to MAX_LEN words)
train_text = train_df['clean'].tolist()
train_text = [' '.join(t.split()[0:MAX_LEN]) for t in train_text]
train_text = np.array(train_text, dtype=object)[:, np.newaxis]
train_label = train_df['label'].tolist()
test_text = test_df['clean'].tolist()
test_text = [' '.join(t.split()[0:MAX_LEN]) for t in test_text]
test_text = np.array(test_text, dtype=object)[:, np.newaxis]
test_label = test_df['label'].tolist()

In [6]:
from keras.engine.topology import Layer
import tensorflow_hub as hub
from keras import backend as K

class ElmoEmbeddingLayer(Layer):
    def __init__(self, **kwargs):
        self.dimensions = 1024
        self.trainable = True
        super(ElmoEmbeddingLayer, self).__init__(**kwargs)
    def build(self, input_shape):
        self.elmo = hub.Module('https://tfhub.dev/google/elmo/3', trainable=self.trainable, name="{}_module".format(self.name))
        self.trainable_weights += tf.trainable_variables(scope="^{}_module/.*".format(self.name))
        super(ElmoEmbeddingLayer, self).build(input_shape)
    def call(self, x, mask=None):
        result = self.elmo(K.squeeze(K.cast(x, tf.string), axis=1),
                      as_dict=True,
                      signature='default',
                      )['default']
        return result
    def compute_mask(self, inputs, mask=None):
        return K.not_equal(inputs, '--PAD--')
    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.dimensions)

Using TensorFlow backend.


In [7]:
from keras import layers
from keras.models import Model
from keras.optimizers import Adam

input_text = layers.Input(shape=(1,), dtype=tf.string)
embedding = ElmoEmbeddingLayer()(input_text)
dense = layers.Dense(1024, activation='relu')(embedding)
pred = layers.Dense(1, activation='sigmoid')(dense)
model = Model(inputs=[input_text], outputs=pred)
model.compile(loss='binary_crossentropy', optimizer=Adam(lr=2e-4), metrics=['accuracy'])
model.summary()

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1)                 0         
_________________________________________________________________
elmo_embedding_layer_1 (Elmo (None, 1024)              4         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              1049600   
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 1025      
Total params: 1,050,629
Trainable params: 1,050,629
Non-trainable params: 0
_________________________________________________________________


In [0]:
from tensorflow.compat.v1 import keras

session = keras.backend.get_session()
init = tf.global_variables_initializer()
session.run(init)

In [0]:
import keras
import numpy as np
import sklearn.metrics as sklm

class Metrics(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.loss = []
        self.precision = []
        self.recall = []
        self.f1s = []
        self.accuracy = []
        self.auc = []

    def on_epoch_end(self, epoch, logs={}):
        score = np.asarray(self.model.predict(self.validation_data[0]))
        predict = np.round(np.asarray(self.model.predict(self.validation_data[0])))
        targ = self.validation_data[1]
        self.loss.append(keras.losses.binary_crossentropy(tf.convert_to_tensor(targ), tf.convert_to_tensor(predict)).eval(session=session).mean())
        self.auc.append(sklm.roc_auc_score(targ, score))
        self.precision.append(sklm.precision_score(targ, predict))
        self.recall.append(sklm.recall_score(targ, predict))
        self.f1s.append(sklm.f1_score(targ, predict))
        self.accuracy.append(sklm.accuracy_score(targ, predict))

        pd.DataFrame({
            'loss': self.loss,
            'precision': self.precision,
            'recall': self.recall,
            'f1s': self.f1s,
            'accuracy': self.accuracy,
            'auc': self.auc
        }).to_csv('recors.csv')

        return

metrics = Metrics()

In [10]:
model.fit(train_text, 
          train_label,
          validation_data=(test_text, test_label),
          epochs=50,
          callbacks=[metrics],
          batch_size=32)

Train on 4562 samples, validate on 1010 samples
Epoch 1/50
4562/4562 [==============================] - 24s 5ms/step - loss: 0.1399 - accuracy: 0.9489 - val_loss: 0.1149 - val_accuracy: 0.9525
Epoch 2/50
4562/4562 [==============================] - 15s 3ms/step - loss: 0.0647 - accuracy: 0.9801 - val_loss: 0.0803 - val_accuracy: 0.9782
Epoch 3/50
4562/4562 [==============================] - 15s 3ms/step - loss: 0.0463 - accuracy: 0.9871 - val_loss: 0.0763 - val_accuracy: 0.9792
Epoch 4/50
4562/4562 [==============================] - 15s 3ms/step - loss: 0.0369 - accuracy: 0.9904 - val_loss: 0.0712 - val_accuracy: 0.9792
Epoch 5/50
4562/4562 [==============================] - 15s 3ms/step - loss: 0.0279 - accuracy: 0.9936 - val_loss: 0.0682 - val_accuracy: 0.9832
Epoch 6/50
4562/4562 [==============================] - 15s 3ms/step - loss: 0.0214 - accuracy: 0.9947 - val_loss: 0.0674 - val_accuracy: 0.9842
Epoch 7/50
4562/4562 [==============================] - 15s 3ms/step - loss: 0.016